In [1]:
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, roc_curve


import warnings
import joblib
import os 
from scipy.sparse import load_npz

warnings.filterwarnings('ignore')

load_dir = '../data/gold/'

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment('fraud_logreg_base_model')


<Experiment: artifact_location='/home/maldu/dscience/projects/fraud_detection/experiments/artifacts_local/1', creation_time=1723547342006, experiment_id='1', last_update_time=1723547342006, lifecycle_stage='active', name='fraud_logreg_base_model', tags={}>

In [3]:
X_train = load_npz(os.path.join(load_dir, 'X_train_scaled.npz'))
X_test = load_npz(os.path.join(load_dir, 'X_test_scaled.npz'))

y_train = joblib.load(os.path.join(load_dir, 'y_train.pkl'))
y_test = joblib.load(os.path.join(load_dir, 'y_test.pkl'))

In [4]:
with mlflow.start_run():
        mlflow.set_tag('developer', 'Maldu')

        # Crear el modelo base
        model = LogisticRegression(max_iter=1000, random_state=42)
        model.fit(X_train, y_train)

        # Evaluar el modelo en el conjunto de entrenamiento
        train_accuracy = model.score(X_train, y_train)
        mlflow.log_metric('train_accuracy', train_accuracy)
        print(f"Training Accuracy: {train_accuracy:.4f}")
        
        # Evaluar el modelo en el conjunto de prueba
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric('accuracy', accuracy)
        print(f"Test Accuracy: {accuracy:.4f}")

        test_accuracy = model.score(X_test, y_test)
        mlflow.log_metric('test_accuracy', test_accuracy)
        print(f"Test Accuracy (using score method): {test_accuracy:.4f}")

        # Confusion Matrix
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()
        print("Confusion Matrix saved and logged.")

        # Classification Report
        report = classification_report(y_test, y_pred, output_dict=True)
        
        # Log precision and recall for each class and average
        for key, value in report.items():
            if isinstance(value, dict):
                for sub_key, sub_value in value.items():
                    if sub_key in ['precision', 'recall']:
                        mlflow.log_param(f'{key}_{sub_key}', sub_value)
                        print(f"{key} - {sub_key}: {sub_value:.4f}")
            else:
                if key in ['precision', 'recall']:
                    mlflow.log_param(f'{key}', value)
                    print(f"{key}: {value:.4f}")

        # ROC and AUC
        y_prob = model.predict_proba(X_test)[:, 1]  # Probabilidades de la clase positiva
        roc_auc = roc_auc_score(y_test, y_prob)
        mlflow.log_metric('roc_auc', roc_auc)
        print(f"ROC AUC: {roc_auc:.4f}")
        
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        plt.savefig("roc_curve.png")
        mlflow.log_artifact("roc_curve.png")
        plt.close()
        print("ROC Curve saved and logged.")

        # Guardar el modelo
        mlflow.sklearn.log_model(model, "model")
        print(f"Model saved. Default artifacts URI: '{mlflow.get_artifact_uri()}'")


Training Accuracy: 0.9983
Test Accuracy: 0.9979
Test Accuracy (using score method): 0.9979
Confusion Matrix saved and logged.
0 - precision: 0.9980
0 - recall: 0.9999
1 - precision: 0.0000
1 - recall: 0.0000
macro avg - precision: 0.4990
macro avg - recall: 0.4999
weighted avg - precision: 0.9960
weighted avg - recall: 0.9979
ROC AUC: 0.9419
ROC Curve saved and logged.


2024/08/13 13:09:38 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/13 13:09:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run bedecked-bird-385 at: http://127.0.0.1:5000/#/experiments/1/runs/4fb50f5fca84406483e07b8b428175b5.
2024/08/13 13:09:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


Model saved. Default artifacts URI: '/home/maldu/dscience/projects/fraud_detection/experiments/artifacts_local/1/4fb50f5fca84406483e07b8b428175b5/artifacts'


Resultados y Análisis

    Precisión y Recall:
        Precisión (Precision) para la clase 0 es alta, pero para la clase 1 es 0. Esto indica que el modelo no está identificando correctamente los ejemplos de la clase 1.
        Recall para la clase 0 es muy alto (casi 1), lo que significa que el modelo está muy bueno identificando ejemplos de la clase 0.
        Recall para la clase 1 es 0, lo que significa que el modelo no está identificando ningún ejemplo de la clase 1.

    Métricas de Evaluación:
        Precisión Macro y Recall Macro: Estos valores están alrededor de 0.5, que reflejan el mal desempeño en la clase minoritaria.
        Precisión Ponderada y Recall Ponderado: Estos valores son relativamente altos, lo que indica que el modelo se comporta bien en la clase mayoritaria, que puede estar sesgando los resultados debido al desbalance de clases.

    ROC AUC:
        El AUC de 0.9419 sugiere que el modelo tiene una buena capacidad para distinguir entre las dos clases. Sin embargo, el alto AUC con las métricas de precisión y recall para la clase 1 en 0 indica que el modelo está fallando en la identificación de ejemplos positivos, lo que puede ser un signo de desbalance de clases.

¿Qué Está Pasando?

El desbalance de clases parece ser el principal problema aquí. La clase 1 (posiblemente la clase de fraude) es mucho menos frecuente en comparación con la clase 0, lo que puede estar causando que el modelo se incline hacia la clase mayoritaria.
¿Qué Puedes Hacer?

    Manejo del Desbalance de Clases:
        Reescalado de Datos: Usa técnicas como sobremuestreo (SMOTE) para la clase minoritaria o submuestreo de la clase mayoritaria.
        Ponderación de Clases: Ajusta el parámetro class_weight en LogisticRegression a 'balanced' para que el modelo preste más atención a la clase minoritaria.

    Ajuste de Hiperparámetros:
        Experimenta con diferentes valores de hiperparámetros como C, penalty, etc.

    Evaluación con Métricas Adecuadas:
        F1 Score: Considera usar el F1 score en lugar de precisión y recall para obtener una métrica que combine ambas.
        Matriz de Confusión: Asegúrate de revisar la matriz de confusión para entender cómo se están clasificando las diferentes clases.